In [22]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
wildfires = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/California_Fire_Incidents.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
wildfires.head(10)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0
5,22992.0,False,Sierra National Forest,NaN,2013,False,/incidents/2013/7/22/aspen-fire/,NaN,NaN,Fresno,...,"217 Aspen Fire, July 2013, Big Creek, Fresno ...",2013-07-22T22:15:00Z,Finalized,NaN,NaN,NaN,NaN,bee8c339-4f26-4b78-a5b4-a8a0ebdb8786,2013-09-24T20:15:00Z,NaN
6,20292.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/8/7/silver-fire/,Firefighters closed the containment lines toda...,Hwy 243 remains closed between Twin Pines Road...,Riverside,...,"Silver Fire, August 2013, Popet Flats Road, Hi...",2013-08-07T14:05:00Z,Finalized,8.0,40.0,NaN,NaN,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,2013-08-12T18:00:00Z,20.0
7,14754.0,False,Klamath National Forest,NaN,2013,False,/incidents/2013/7/31/salmon-river-complex/,NaN,NaN,Siskiyou,...,"210 Salmon River Complex, August 2013, July 20...",2013-07-31T22:00:00Z,Finalized,NaN,NaN,NaN,NaN,ba76c009-09c9-497c-b923-1ac8cf05c20d,2013-08-31T06:45:00Z,NaN
8,12503.0,False,Six Rivers National Forest,NaN,2013,False,/incidents/2013/8/10/corral-complex/,NaN,NaN,Humboldt,...,"Corral Complex, August 2013, Tish Tang Ridge, ...",2013-08-10T11:40:00Z,Finalized,NaN,NaN,NaN,NaN,f3dcbca8-f8ed-46d0-8f6e-9c7fc2fb03e6,2013-08-12T12:00:00Z,NaN
9,11429.0,False,CAL FIRE Tehama-Glenn Unit,NaN,2013,True,/incidents/2013/8/23/deer-fire/,Fire suppression repair will continue througho...,NaN,Tehama,...,"Deer Fire, August 2013, Tehama County, Deer Cr...",2013-08-23T14:15:00Z,Finalized,NaN,NaN,NaN,NaN,956dbcf6-db40-4b61-b235-4ede14738f1e,2013-08-29T16:45:00Z,8.0


In [24]:
#Print the shape of the data
wildfires.shape

(1636, 40)

In [25]:
#Use the pandas info function to get an overview of the different features within the dataset and their datatypes as well as NaN counts
wildfires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      111 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [26]:
wildfires.drop(['Active', 'CanonicalUrl', 'ConditionStatement', 'ControlStatement' ,'Extinguished', 'Featured',
                'Final', 'Location', 'PercentContained', 'Public', 'SearchDescription',
                'Status', 'UniqueId', 'Updated'], axis = 1, inplace = True)

In [27]:
wildfires.head(10)

,AcresBurned,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,Counties,CountyIds,CrewsInvolved,Dozers,Engines,...,MajorIncident,Name,PersonnelInvolved,SearchKeywords,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders
0,257314.0,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,Tuolumne,55,NaN,NaN,NaN,...,False,Rim Fire,NaN,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,NaN,NaN,NaN,NaN,NaN
1,30274.0,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,Los Angeles,19,NaN,NaN,NaN,...,False,Powerhouse Fire,NaN,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,NaN,NaN,NaN,NaN,NaN
2,27531.0,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,Riverside,33,NaN,NaN,NaN,...,False,Mountain Fire,NaN,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,NaN,NaN,NaN,NaN,NaN
3,27440.0,Tahoe National Forest,NaN,2013,False,Placer,31,NaN,NaN,NaN,...,False,American Fire,NaN,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,NaN,NaN,NaN,NaN,NaN
4,24251.0,Ventura County Fire/CAL FIRE,NaN,2013,True,Ventura,56,47.0,8.0,117.0,...,True,Springs Fire,2167.0,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,6.0,10.0,NaN,NaN,11.0
5,22992.0,Sierra National Forest,NaN,2013,False,Fresno,10,NaN,NaN,NaN,...,False,Aspen Fire,NaN,"217 Aspen Fire, July 2013, Big Creek, Fresno ...",2013-07-22T22:15:00Z,NaN,NaN,NaN,NaN,NaN
6,20292.0,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,Riverside,33,63.0,20.0,201.0,...,True,Silver Fire,2106.0,"Silver Fire, August 2013, Popet Flats Road, Hi...",2013-08-07T14:05:00Z,8.0,40.0,NaN,NaN,20.0
7,14754.0,Klamath National Forest,NaN,2013,False,Siskiyou,47,NaN,NaN,NaN,...,False,Salmon River Complex,NaN,"210 Salmon River Complex, August 2013, July 20...",2013-07-31T22:00:00Z,NaN,NaN,NaN,NaN,NaN
8,12503.0,Six Rivers National Forest,NaN,2013,False,Humboldt,12,NaN,NaN,NaN,...,False,Corral Complex,NaN,"Corral Complex, August 2013, Tish Tang Ridge, ...",2013-08-10T11:40:00Z,NaN,NaN,NaN,NaN,NaN
9,11429.0,CAL FIRE Tehama-Glenn Unit,NaN,2013,True,Tehama,52,30.0,3.0,36.0,...,True,Deer Fire,898.0,"Deer Fire, August 2013, Tehama County, Deer Cr...",2013-08-23T14:15:00Z,NaN,NaN,NaN,NaN,8.0
